# pyCrypto

## From PIP documentation:

In [1]:
from Crypto.Hash import SHA256

In [2]:
hash = SHA256.new()

In [3]:
hash.update(b'message')

In [4]:
hash.digest()

b'\xabS\n\x13\xe4Y\x14\x98+y\xf9\xb7\xe3\xfb\xa9\x94\xcf\xd1\xf3\xfb"\xf7\x1c\xea\x1a\xfb\xf0+F\x0cm\x1d'

In [5]:
from Crypto.Cipher import AES
obj = AES.new('This is a key123', AES.MODE_CBC, 'This is an IV456')
message = "The answer is no"
ciphertext = obj.encrypt(message)
ciphertext

b'\xd6\x83\x8dd!VT\x92\xaa`A\x05\xe0\x9b\x8b\xf1'

In [6]:
obj2 = AES.new('This is a key123', AES.MODE_CBC, 'This is an IV456')
obj2.decrypt(ciphertext)

b'The answer is no'

As of PyCrypto 2.1.0, PyCrypto provides an easy-to-use random number generator:

In [7]:
from Crypto import Random
rndfile = Random.new()
rndfile.read(16)

b'\x19\x88{\xc1\x99.\xda\xd2\x81/\x04\x1a\xd3\xee\x94\xc5'

A stronger version of Python’s standard “random” module is also provided:

In [8]:
from Crypto.Random import random
random.choice(['dogs', 'cats', 'bears'])

'cats'

## Tutorial

### Hash functions
* It should be very difficult to guess the input string based on the output string.
* It should be very difficult to find 2 different input strings having the same hash output.
* It should be very difficult to modify the input string without modifying the output hash value.

### Public-key algorithms

One disadvantage with the encryption algorithms seen above is that both sides need to know the key. With public-key algorithms, there are two different keys: one to encrypt and one to decrypt. You only need to share the encryption key and only you can decrypt the message with your private decryption key.

#### Public/private key pair

It is easy to generate a private/public key pair with pycrypto. We need to specify the size of the key in bits: we picked 1024 bits. Larger is more secure. We also need to specify a random number generator function, we use the Random module of pycrypto for that.

In [41]:
from Crypto.PublicKey import RSA
from Crypto import Random

random_generator = Random.new().read
key = RSA.generate(1024, random_generator)
key

<_RSAobj @0x7fe57692c048 n(1024),e,d,p,q,u,private>

#### Encrypt

Now that we have our key pair, we can encrypt some data. First, we extract the public key from the key pair and use it to encrypt some data. 32 is a random parameter used by the RSA algorithm to encrypt the data. This step simulates us publishing the encryption key and someone using it to encrypt some data before sending it to us.

In [10]:
public_key = key.publickey()
enc_data = public_key.encrypt(b'abcdefgh', 32)
enc_data


(b'g\x9a\xcd\x8c\xaf\x9c\xab%\xe2\xd7>e\xb1\xc2\xe9\xc47 q\x83\xe9\x1f\xde\x96\xa1z\xca[\x98\xa8\x7f)\x94\xf6T\xa3`\r\xccw\x8e\x82N"\x1938\xc3\x11\xba=1A\xf2?\x1cs\xbf\xe2\\\x98C%\xd7\xf4h_\xecy;HVV\xbf}(\x95\xf2\xfe\xab!\xb4P\xbb+h\xfa?f\xec\x98\x9b"\x95\xdd?\xa0\x06\x8bd\x9caxTx\x13i\xd5o\x13M\xbee\x8d\xaeN\n\x1f\xb4\x07\xd83\xfd\x88\x00\xcc\xfe\x05',)

Decrypt

In [11]:
key.decrypt(enc_data)

b'abcdefgh'

#### Sign

Signing a message can be useful to check the author of a message and make sure we can trust its origin. Next is an example on how to sign a message. The hash for this message is calculated first and then passed to the sign() method of the RSA key. You can use other algorithms like DSA or ElGamal.

In [12]:
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto import Random

key = RSA.generate(1024, random_generator)

text = b'abcdefgh'

hash = SHA256.new(text).digest()

hash


b'\x9cV\xccQ\xb3t\xc3\xba\x18\x92\x10\xd5\xb6\xd4\xbfWy\r5\x1c\x96\xc4|\x02\x19\x0e\xcf\x1eC\x065\xab'

In [13]:
signature = key.sign(hash, '')

#### Verify

Knowing the public key, it is easy to verify a message. The plain text is sent to the user along with the signature. The receiving side calculates the hash value and then uses the public key verify() method to validate its origin.

In [14]:
text = b'abcdefgh'
hash = SHA256.new(text).digest()
public_key = key.publickey()
public_key.verify(hash, signature)


True

# MY EXPERIMENTS

In [16]:
import pickle

In [21]:
with open('RSA_test', 'wb') as file:
    pickle.dump(key, file)

In [22]:
with open('RSA_test', 'rb') as file:
    new_kew = pickle.load(file)

In [24]:
new_kew

<_RSAobj @0x7fe57692c208 n(1024),e,d,p,q,u,private>

In [37]:
test = {'a': 1, 'b': 2}
with open('RSA_test', 'wb') as file:
    pickle.dump(test, file)


In [39]:
from pathlib import Path
file = Path('RSA_test').read_bytes()

dicts = pickle.loads(file)

In [33]:
c = key.decrypt(enc_data)

In [34]:
dicts = pickle.loads(c)

In [40]:
dicts

{'a': 1, 'b': 2}

In [11]:
from Crypto import Random
from Crypto.Cipher import AES
key = b'admin'
password = b'super_secret_password'

key = key + (16 - len(key) % 16) * b'0'
len(key)

16

In [12]:
i_vector = Random.new().read(AES.block_size)
cipher = AES.new(key, AES.MODE_CFB, i_vector)

In [13]:
data = cipher.encrypt(password)

In [14]:
data

b't\xab\xb3\xad\xe6\xb79\xbb\x96\x98\xcb\x1f-\xba\x87qRI@\xce\xd2'